In [1]:
import numpy as np

# 1- Calculate f(x) = x^2 - xy + y^2 + 2x + 4y + 3

## for the food where food = [x, y]

In [2]:
def calc_food_fx(food):
    x, y = food
    return x ** 2 - x * y + y ** 2 + 2 * x + 4 * y + 3

# 2- Calculate food fitness 

In [3]:
def calc_food_fitness(fx):
    if fx >= 0:
        return 1 / (1 + fx)
    else:
         return 1 + abs(fx)

# 3- Calculate f(x) for all the population

In [4]:
def calc_swarm_fx(food_source):
    fx = np.empty(food_source.shape[0])
    for index, food in enumerate(food_source):
        fx[index] =  calc_food_fx(food)
    return fx

# 4- Calculate fitness for all the population

In [5]:
def calc_swarm_fitness(fx):
    fitness = np.empty(fx.shape)
    for index, f in enumerate(fx):
        fitness[index] = calc_food_fitness(f)     
    return fitness

# 5- Employee Phase 

In [6]:
def employeePhase(food_source, fx, fitness, trial):
    
    new_food_source = np.zeros(food_source.shape)
    new_fx = np.zeros(fx.shape)
    new_fitness = np.zeros(fitness.shape)
    
    for index, row in enumerate(food_source):
        
        x_index = np.random.randint(food_source.shape[1]) 
        partner_index = np.random.randint(food_source.shape[0])
        partner = food_source[partner_index]
        
        # x_new = x + phi * (x - x_partner)
        phi = np.random.random()
        food = row.copy()
        food[x_index] = food[x_index] + phi * (food[x_index] - partner[x_index])
        
        # Scaling out of the range value to (-5, 5)
        if food[x_index] > 5.0:
            food[x_index] = 5.0
        if food[x_index] < -5.0:
            food[x_index] = -5.0
            
        food_fx = calc_food_fx(food)    
        food_fitness = calc_food_fitness(food_fx)
        if food_fitness < fitness[index]: # the new solution better than old solution
            new_food_source[index] = food
            new_fx[index] = calc_food_fx(food)
            new_fitness[index] = calc_food_fitness(food_fx)
            trial[index] = 0
            
        else: # the old solution better than the new
            
            new_food_source[index] = food_source[index]
            new_fx[index] = fx[index]
            new_fitness[index] = fitness[index]
            trial[index] += 1    
            
    return new_food_source, new_fx, new_fitness, trial

# 6- OnLooker Phase

In [7]:
def onLookerPhase(food_source, fx, fitness, trial):
    
    np.seterr(invalid='ignore') # Supress/hide the warning
    probs = fitness / fitness.sum() # calculate the population probability
    
    new_food_source = np.zeros(food_source.shape)
    new_fx = np.zeros(fx.shape)
    new_fitness = np.zeros(fitness.shape)
    
    for index, row in enumerate(food_source):
        
        r = np.random.random()
        if r > probs[index]:
            new_food_source[index] = food_source[index]
            new_fx[index] = fx[index]
            new_fitness[index] = fitness[index]
            continue
        
        x_index = np.random.randint(food_source.shape[1]) 
        partner_index = np.random.randint(food_source.shape[0])
        partner = food_source[partner_index]
        
        # x_new = x + phi * (x - x_partner)
        food = row.copy()
        phi = np.random.random()
        food[x_index] = food[x_index] + phi * (food[x_index] - partner[x_index])
        
        # Scaling out of the range value to (-5, 5)
        if food[x_index] > 5.0:
            food[x_index] = 5.0 
        if food[x_index] < -5.0:
            food[x_index] = -5.0
            
        food_fx = calc_food_fx(food)    
        food_fitness = calc_food_fitness(food_fx)
        if food_fitness < fitness[index]: # the new solution better than old solution
            new_food_source[index] = food
            new_fx[index] = calc_food_fx(food)
            new_fitness[index] = calc_food_fitness(food_fx)
            trial[index] = 0
            
        else: # the old solution better than the new
            
            new_food_source[index] = food_source[index]
            new_fx[index] = fx[index]
            new_fitness[index] = fitness[index]
            trial[index] += 1    
            
    return new_food_source, new_fx, new_fitness, trial

# 7- Scout Phase

In [8]:
def scoutPhase(food_source, fx, fitness, trial, scope, limit):
    
    (b, a) = scope #(-5, 5)
    
    for index in range(food_source.shape[0]):
        
        if trial[index] > limit:
            
            food_source[index] = (b - a) * np.random.random((1, food_source.shape[1])) + a
            fx[index] = calc_food_fx(food_source[index])
            fitness[index] = calc_food_fitness(fx[index])
            trial[index] = 0
            
    return food_source, fx, fitness, trial

# 8- ABC algorithm for solve the equation

In [9]:
def ABC(swarm_size = 10, dim = 2, limit = 1, iteration = 20):
    # initialize the population
    (a, b) = (-5, 5)
    food_source = (b - a) * np.random.random((swarm_size, dim)) + a
    fx = np.zeros(10)
    fitness = np.zeros(10)
    trial = np.zeros(10)
    
    best_food = None
    best_fx = None
    
    for _ in range(iteration):
        
        food_source, fx, fitness, trial = employeePhase(food_source, fx, fitness, trial) # Employee Phase
        
        food_source, fx, fitness, trial = onLookerPhase(food_source, fx, fitness, trial) # OnLooker Phase
        
        food_source, fx, fitness, trial = scoutPhase(food_source, fx, fitness, trial, (a, b), limit) # Scout Phase
        
        # Memorize the best solution
        i = fx.argmax()
        new_best_fx = fx[i]
        if best_fx is None or best_fx < new_best_fx:
            best_fx = new_best_fx
            best_food = food_source[i]
        
    return best_food, best_fx

# 9- Test the algorithm

In [10]:
food, fx = ABC(swarm_size = 10, dim = 2, limit = 1, iteration = 20)
print(f"the best solution is\nX = {food[0]}\nY = {food[1]}")
print(f"f(x) = x^2 - xy + y^2 + 2x + 4y + 3 = {fx}")

the best solution is
X = -5.0
Y = 5.0
f(x) = x^2 - xy + y^2 + 2x + 4y + 3 = 88.0
